# **LSTM Stock Prediction**
- **Time series** data is just data points arranged in a sequence over time, e.g. Tracking stock prices daily.
- It’s a powerful way to see how things change over days, months, or even years.
- This kind of data is everywhere—think about **tracking daily temperatures, website traffic, or even your daily steps**!
- In this, let’s dive into time series data using popular tech stocks: **Apple, Amazon, Google, and Microsoft**. We’ll use Python tools to:

  - **Fetch Stock Data**: We’ll use **yfinance**, a Python library, to easily pull historical stock data.
  - **Visualize Trends**: Using Matplotlib and Seaborn to create visuals, we’ll look at how stock prices change over time.
  - **Analyze Stock Risk**: We’ll check how "risky" each stock is based on past performance, which can hint at future behavior.
  - **Predict Future Prices**: Using a machine learning approach called **LSTM (Long Short-Term Memory)**, we’ll predict future stock prices. LSTM is great for time-based predictions because it remembers patterns over time.

# **Importing Libraries**

In [3]:
from pandas_datareader.data import DataReader
from pandas_datareader import data as pdr
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import pandas as pd
import numpy as np


sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")

# **Collecting & Downloading Data**

In [6]:
# Function to download stock data
def download_stock_data(stock_list, start_date, end_date):
    stock_data = {}
    for stock in stock_list:
        stock_data[stock] = yf.download(stock, start=start_date, end=end_date, )
    return stock_data

# Function to add company names to the data
def add_company_names(stock_data, company_names):
    for stock, name in zip(stock_data.keys(), company_names):
        stock_data[stock]["Company_Name"] = name
    return stock_data

# Define the list of tech stocks and company names
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']
company_names = ["APPLE", "GOOGLE", "MICROSOFT", "AMAZON"]

# Set the date range for the last year
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Download stock data and add company names
stock_data = download_stock_data(tech_list, start_date, end_date)
stock_data_with_names = add_company_names(stock_data, company_names)

# Combine all data into a single DataFrame
df = pd.concat(stock_data_with_names.values(), axis=0)

# Display the last 10 rows of the DataFrame
df.tail(10)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Price                     Adj Close Close High  Low Open Volume Company_Name  \
Ticker                         AAPL  AAPL AAPL AAPL AAPL   AAPL                
Date                                                                           
2024-10-15 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-16 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-17 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-18 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-21 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-22 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-23 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-24 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-25 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   
2024-10-28 00:00:00+00:00       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON   

Price                     Adj Close Close High  ...       Low Open Volume  \
Ticker                         GOOG  GOOG GOOG  ... MSFT MSFT MSFT   MSFT   
Date                                            ...                         
2024-10-15 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-16 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-17 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-18 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-21 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-22 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-23 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-24 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-25 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   
2024-10-28 00:00:00+00:00       NaN   NaN  NaN  ...  NaN  NaN  NaN    NaN   

Price                       Adj Close       Close        High         Low  \
Ticker                           AMZN        AMZN        AMZN        AMZN   
Date                                                                        
2024-10-15 00:00:00+00:00  187.690002  187.690002  188.410004  184.580002   
2024-10-16 00:00:00+00:00  186.889999  186.889999  187.779999  185.610001   
2024-10-17 00:00:00+00:00  187.529999  187.529999  188.940002  186.000000   
2024-10-18 00:00:00+00:00  188.990005  188.990005  190.740005  186.279999   
2024-10-21 00:00:00+00:00  189.070007  189.070007  189.460007  186.399994   
2024-10-22 00:00:00+00:00  189.699997  189.699997  191.520004  186.979996   
2024-10-23 00:00:00+00:00  184.710007  184.710007  189.160004  183.690002   
2024-10-24 00:00:00+00:00  186.380005  186.380005  187.110001  183.860001   
2024-10-25 00:00:00+00:00  187.830002  187.830002  190.449997  187.529999   
2024-10-28 00:00:00+00:00  189.425003  189.425003  190.210007  188.830002   

Price                            Open      Volume  
Ticker                           AMZN        AMZN  
Date                                               
2024-10-15 00:00:00+00:00  187.630005  32178900.0  
2024-10-16 00:00:00+00:00  187.050003  23456800.0  
2024-10-17 00:00:00+00:00  188.220001  25039400.0  
2024-10-18 00:00:00+00:00  187.149994  37417700.0  
2024-10-21 00:00:00+00:00  188.050003  24639400.0  
2024-10-22 00:00:00+00:00  188.350006  29650600.0  
2024-10-23 00:00:00+00:00  188.850006  31937100.0  
2024-10-24 00:00:00+00:00  185.250000  21647400.0  
2024-10-25 00:00:00+00:00  187.850006  29310400.0  
2024-10-28 00:00:00+00:00  189.580002  10686429.0  

[10 rows x 25 columns]

# **Data Pre-Processing**